# LoRA (PEFT) でTTSモデルを軽量ファインチューニング（例）

このノートブックは、Hugging FaceのPEFTを使ってLoRAをTTSモデルへ適用する最小構成の例です。
対象モデルやデータセットの前提に合わせて適宜調整してください。


## 1. 依存関係のインストール

ColabのCPUランタイムで実行する想定です。


In [1]:
!pip install -q peft transformers datasets accelerate


## 2. ベースモデルのロードとLoRAの適用

ここではプレースホルダとして`base_model`を用意し、LoRAを適用する流れを示します。
実際のTTSモデルクラスに合わせて読み替えてください。


In [4]:
from peft import LoraConfig, get_peft_model

# 例: base_model = ...（VITS/Tortoiseなどのロード処理をここに記載）
base_model = None  # TODO: 実際のモデルに差し替えてください

config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",  # モデルに合わせて調整
)

model = get_peft_model(base_model, config)
model.print_trainable_parameters()


AttributeError: 'NoneType' object has no attribute '__dict__'

## 3. データセット準備

- テキストと音声のペアを用意します。
- サンプリングレートの統一、無音除去、文字起こし修正などの前処理を実施します。


In [ ]:
from datasets import load_dataset

# 例: ローカルCSVやHF Hubのデータセットを読み込む
# dataset = load_dataset("path/to/your_dataset")
# dataset = dataset.train_test_split(test_size=0.05)

# TODO: データセットのスキーマに合わせて前処理を実装してください


## 4. トレーナーで学習

`Trainer`にモデル・データセット・学習設定を渡して学習します。
TTSモデルによっては独自のトレーナー実装が必要な場合があります。


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="outputs/lora_tts",
    per_device_train_batch_size=1,
    learning_rate=1e-4,
    num_train_epochs=5,
    no_cuda=True,
    logging_steps=50,
    save_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,  # TODO: dataset["train"]を設定
    eval_dataset=None,   # TODO: dataset["test"]を設定
)

trainer.train()


## 5. LoRA重みの保存

学習後はLoRA重みのみを保存し、配布・適用を容易にします。


In [ ]:
model.save_pretrained("outputs/lora_tts/lora_weights")


## 参考

- Hugging Face PEFT ドキュメント
- VITS / Tortoise 公式実装
